Global setup



In [16]:
n=1
k=2
q=23 #AA
eta=2
R.<x> = ZZ[]

Bob generates a key


In [17]:
A = matrix([[randrange(0,q), randrange(0,q)], 
            [randrange(0,q), randrange(0,q)]])
A

[18 10]
[16  4]

In [18]:
latex(A)

\left(\begin{array}{rr}
18 & 10 \\
16 & 4
\end{array}\right)

In [19]:
def binomial_centered():
    a1=randrange(0,2)
    a2=randrange(0,2)
    b1=randrange(0,2)
    b2=randrange(0,2)
    if eta == 2: 
        return a1+a2-b1-b2
    elif eta == 1:
        return a1-b1
    else:
        return 0

In [28]:
s = matrix([[binomial_centered()], 
            [binomial_centered()]])
s

[1]
[2]

In [29]:
latex(s)

\left(\begin{array}{r}
1 \\
2
\end{array}\right)

$s$ is the private key


In [34]:
e = matrix([[binomial_centered()], 
            [binomial_centered()]])
e

[ 0]
[-1]

In [35]:
latex(e)

\left(\begin{array}{r}
0 \\
-1
\end{array}\right)

In [36]:
t = matrix(((A*s + e) % (x^n+1) % q))
t

[15]
[ 0]

In [37]:
latex(t)

\left(\begin{array}{r}
15 \\
0
\end{array}\right)

$(A,t)$ is the public key


Alice encodes her plaintext



In [41]:
plist = (ord("A") - ord("A") + 1).binary()
p = R(list(reversed(plist)))
p

1

In [42]:
r = matrix([[binomial_centered()], 
            [binomial_centered()]])
r

[-1]
[-1]

In [43]:
latex(r)

\left(\begin{array}{r}
-1 \\
-1
\end{array}\right)

In [44]:
e1 = matrix([[binomial_centered()],
             [binomial_centered()]])
e1

[0]
[0]

In [45]:
latex(e1)

\left(\begin{array}{r}
0 \\
0
\end{array}\right)

In [46]:
e2 = binomial_centered()
e2

0

In [47]:
u = (A.transpose()*r+e1) % (x^n+1) % q
u

[12]
[ 9]

In [48]:
latex(u)

\left(\begin{array}{r}
12 \\
9
\end{array}\right)

In [49]:
t.transpose()*r % (x^n+1) % q

[8]

In [50]:
round(q/2)*p

12

In [51]:
v = (t.transpose()*r+e2 + round(q/2)*p) % (x^n+1) % q
v

[-3]

$\mathbf{u}$ and $v$ is the ciphertext that Alice sends to Bob



In [52]:
(v-s.transpose()*u) % (x^n+1) % q

[13]

In [53]:
(round(q/2)^(-1)*((v-s.transpose()*u) % (x^n+1) % q))

[13/12]

In [54]:
pprime = (round(q/2)^(-1)*((v-s.transpose()*u) % (x^n+1) % q))[0,0].map_coefficients(round, new_base_ring=Zmod(2)) ###UGH***
pprime

1

In [55]:
p

1

In [56]:
p == pprime

True

In [57]:
I2 = identity_matrix(2)
I2

[1 0]
[0 1]

In [58]:
M = block_matrix([[A.transpose()], [I2], [-t.transpose()], [q*I2]])
M

[ 18  16]
[ 10   4]
[-------]
[  1   0]
[  0   1]
[-------]
[-15   0]
[-------]
[ 23   0]
[  0  23]

In [62]:
latex(M)

\left(\begin{array}{rr}
18 & 16 \\
10 & 4 \\
\hline
 1 & 0 \\
0 & 1 \\
\hline
 -15 & 0 \\
\hline
 23 & 0 \\
0 & 23
\end{array}\right)

In [63]:
latex(identity_matrix(7))

\left(\begin{array}{rrrrrrr}
1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1
\end{array}\right)

In [59]:
M1 = block_matrix([[M, identity_matrix(7)]])
M1

[ 18  16|  1   0   0   0   0   0   0]
[ 10   4|  0   1   0   0   0   0   0]
[  1   0|  0   0   1   0   0   0   0]
[  0   1|  0   0   0   1   0   0   0]
[-15   0|  0   0   0   0   1   0   0]
[ 23   0|  0   0   0   0   0   1   0]
[  0  23|  0   0   0   0   0   0   1]

In [64]:
sage_input(M1)


matrix(ZZ, [[18, 16, 1, 0, 0, 0, 0, 0, 0], [10, 4, 0, 1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 1, 0, 0, 0], [-15, 0, 0, 0, 0, 0, 1, 0, 0], [23, 0, 0, 0, 0, 0, 0, 1, 0], [0, 23, 0, 0, 0, 0, 0, 0, 1]])

In [60]:
M1.hermite_form()

[ 1  0  0  0  0  0  3  2  0]
[ 0  1  0  0  0  1  0  0  0]
[ 0  0  1  0  0  7 15  9 -1]
[ 0  0  0  1  0 19 16 10 -1]
[ 0  0  0  0  1  0 20 13  0]
[ 0  0  0  0  0 23  0  0 -1]
[ 0  0  0  0  0  0 23 15  0]